In [1]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [2]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None]) #Si puede haber nulos
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [3]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data, este data es 
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
data.head().collect()

car_id,timestamp,latitude,longitude,notification
str,str,f64,f64,str
"""car_0""","""2023-01-01T00:00:00""",25.66055,-94.37004,"""low_fuel"""
"""car_0""","""2023-01-01T00:17:04.658414""",25.66055,-94.37004,null
"""car_0""","""2023-01-01T00:04:09.063887""",25.66055,-94.37004,"""low_fuel"""
"""car_0""","""2023-01-01T00:03:41.496538""",25.66055,-94.37004,null
"""car_0""","""2023-01-01T00:03:52.305937""",25.66055,-94.37004,null


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [4]:
data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, format = "%Y-%m-%dT%H:%M:%S.%f", strict=False).alias("timestamp"))

df = data.collect()
print(df.head())

/tmp/ipykernel_43369/2386627681.py:1: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, format = "%Y-%m-%dT%H:%M:%S.%f", strict=False).alias("timestamp"))


shape: (5, 5)
┌────────┬────────────────────────────┬──────────┬───────────┬──────────────┐
│ car_id ┆ timestamp                  ┆ latitude ┆ longitude ┆ notification │
│ ---    ┆ ---                        ┆ ---      ┆ ---       ┆ ---          │
│ str    ┆ datetime[μs]               ┆ f64      ┆ f64       ┆ str          │
╞════════╪════════════════════════════╪══════════╪═══════════╪══════════════╡
│ car_0  ┆ null                       ┆ 25.66055 ┆ -94.37004 ┆ low_fuel     │
│ car_0  ┆ 2023-01-01 00:17:04.000658 ┆ 25.66055 ┆ -94.37004 ┆ null         │
│ car_0  ┆ 2023-01-01 00:04:09.000063 ┆ 25.66055 ┆ -94.37004 ┆ low_fuel     │
│ car_0  ┆ 2023-01-01 00:03:41.000496 ┆ 25.66055 ┆ -94.37004 ┆ null         │
│ car_0  ┆ 2023-01-01 00:03:52.000305 ┆ 25.66055 ┆ -94.37004 ┆ null         │
└────────┴────────────────────────────┴──────────┴───────────┴──────────────┘


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt


USA ROLLING

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [5]:
data = data.with_columns( 
    pl.col("timestamp").diff().cast(pl.Int64).over("car_id").alias("notification_time")
)

df = data.collect()
print(df.head())

shape: (5, 6)
┌────────┬────────────────────────────┬──────────┬───────────┬──────────────┬───────────────────┐
│ car_id ┆ timestamp                  ┆ latitude ┆ longitude ┆ notification ┆ notification_time │
│ ---    ┆ ---                        ┆ ---      ┆ ---       ┆ ---          ┆ ---               │
│ str    ┆ datetime[μs]               ┆ f64      ┆ f64       ┆ str          ┆ i64               │
╞════════╪════════════════════════════╪══════════╪═══════════╪══════════════╪═══════════════════╡
│ car_0  ┆ null                       ┆ 25.66055 ┆ -94.37004 ┆ low_fuel     ┆ null              │
│ car_0  ┆ 2023-01-01 00:17:04.000658 ┆ 25.66055 ┆ -94.37004 ┆ null         ┆ null              │
│ car_0  ┆ 2023-01-01 00:04:09.000063 ┆ 25.66055 ┆ -94.37004 ┆ low_fuel     ┆ -775000595        │
│ car_0  ┆ 2023-01-01 00:03:41.000496 ┆ 25.66055 ┆ -94.37004 ┆ null         ┆ -27999567         │
│ car_0  ┆ 2023-01-01 00:03:52.000305 ┆ 25.66055 ┆ -94.37004 ┆ null         ┆ 10999809          │
└─────

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [8]:
!pip install geopy


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 752.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 KB 6.0 MB/s eta 0:00:00


In [11]:
from geopy.distance import geodesic
def calculaDistance (lat0, lon0, lat, lon):
    if lat0 is None or lon0 is None:
        return None 
    return geodesic((lat0, lon0), (lat, lon)).meters

data = data.with_columns([
    pl.col("latitude").shift(1).over("car_id").alias("latitude0"), 
    pl.col("longitude").shift(1).over("car_id").alias("longitude0")
    ]).with_columns(
        pl.struct(["latitude0", "longitude0", "latitude", "longitude"])
        .map_elements(lambda row: calculaDistance(row["latitude0"], row["longitude0"], row["latitude"], row["longitude"]))
                      .alias("distance")
    )
    
df = data.collect()
print(df.head())

/tmp/ipykernel_43369/3516352480.py:16: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = data.collect()


shape: (5, 9)
┌────────┬─────────────┬──────────┬───────────┬───┬────────────┬───────────┬────────────┬──────────┐
│ car_id ┆ timestamp   ┆ latitude ┆ longitude ┆ … ┆ notificati ┆ latitude0 ┆ longitude0 ┆ distance │
│ ---    ┆ ---         ┆ ---      ┆ ---       ┆   ┆ on_time    ┆ ---       ┆ ---        ┆ ---      │
│ str    ┆ datetime[μs ┆ f64      ┆ f64       ┆   ┆ ---        ┆ f64       ┆ f64        ┆ f64      │
│        ┆ ]           ┆          ┆           ┆   ┆ i64        ┆           ┆            ┆          │
╞════════╪═════════════╪══════════╪═══════════╪═══╪════════════╪═══════════╪════════════╪══════════╡
│ car_0  ┆ null        ┆ 25.66055 ┆ -94.37004 ┆ … ┆ null       ┆ null      ┆ null       ┆ null     │
│ car_0  ┆ 2023-01-01  ┆ 25.66055 ┆ -94.37004 ┆ … ┆ null       ┆ 25.66055  ┆ -94.37004  ┆ 0.0      │
│        ┆ 00:17:04.00 ┆          ┆           ┆   ┆            ┆           ┆            ┆          │
│        ┆ 0658        ┆          ┆           ┆   ┆            ┆           ┆ 

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [12]:
# Validar esquema inicial
print("Esquema inicial:")
print(data.schema)

# Inspeccionar valores en columnas problemáticas
try:
    print("Datos de 'notification_time':")
    print(data.select("notification_time").collect())
except Exception as e:
    print(f"Error en 'notification_time': {e}")

try:
    print("Datos de 'distance':")
    print(data.select("distance").collect())
except Exception as e:
    print(f"Error en 'distance': {e}")

try:
    print("Datos de 'timestamp':")
    print(data.select("timestamp").collect())
except Exception as e:
    print(f"Error en 'timestamp': {e}")

Esquema inicial:
Schema([('car_id', String), ('timestamp', Datetime(time_unit='us', time_zone=None)), ('latitude', Float64), ('longitude', Float64), ('notification', String), ('notification_time', Int64), ('latitude0', Float64), ('longitude0', Float64), ('distance', Unknown)])
Datos de 'notification_time':


/tmp/ipykernel_43369/2319535041.py:3: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print(data.schema)


shape: (6_297_662, 1)
┌───────────────────┐
│ notification_time │
│ ---               │
│ i64               │
╞═══════════════════╡
│ null              │
│ null              │
│ -775000595        │
│ -27999567         │
│ 10999809          │
│ …                 │
│ 137999717         │
│ 837000007         │
│ -1339000529       │
│ 1783000597        │
│ 317999556         │
└───────────────────┘
Datos de 'distance':


/tmp/ipykernel_43369/2319535041.py:14: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  print(data.select("distance").collect())


shape: (6_297_662, 1)
┌─────────────┐
│ distance    │
│ ---         │
│ f64         │
╞═════════════╡
│ null        │
│ 0.0         │
│ 0.0         │
│ 0.0         │
│ 0.0         │
│ …           │
│ 0.0         │
│ 0.0         │
│ 0.0         │
│ 1009.896162 │
│ 0.0         │
└─────────────┘
Datos de 'timestamp':
shape: (6_297_662, 1)
┌────────────────────────────┐
│ timestamp                  │
│ ---                        │
│ datetime[μs]               │
╞════════════════════════════╡
│ null                       │
│ 2023-01-01 00:17:04.000658 │
│ 2023-01-01 00:04:09.000063 │
│ 2023-01-01 00:03:41.000496 │
│ 2023-01-01 00:03:52.000305 │
│ …                          │
│ 2023-01-07 23:17:26.000649 │
│ 2023-01-07 23:31:23.000656 │
│ 2023-01-07 23:09:04.000127 │
│ 2023-01-07 23:38:47.000724 │
│ 2023-01-07 23:44:05.000280 │
└────────────────────────────┘


In [24]:
# 3. Crear intervalos de `x` minutos por carro y calcular estadísticas
x = 5

# Crear una nueva columna que indique el intervalo de tiempo al que pertenece cada notificación
data = data.with_columns(
    (pl.col("timestamp").dt.truncate(f'{x}m')).alias("time_interval")
)

# Agrupar por `car_id` y `time_interval` y calcular estadísticas para `notification_time` y `distance`
# Utilizar group_by con las columnas car_id y time_interval
data = data.group_by(["car_id", "time_interval"]).agg([
    pl.col("notification_time").mean().alias("notification_time_mean"),
    pl.col("notification_time").median().alias("notification_time_median"),
    pl.col("notification_time").var().alias("notification_time_var"),
    pl.col("notification_time").max().alias("notification_time_max"),
    pl.col("notification_time").min().alias("notification_time_min"),
    
    pl.col("distance").mean().alias("distance_mean"),
    pl.col("distance").median().alias("distance_median"),
    pl.col("distance").var().alias("distance_var"),
    pl.col("distance").max().alias("distance_max"),
    pl.col("distance").min().alias("distance_min"),
])

# Mostrar algunas filas del LazyFrame limpio para validar
df = data.collect()
print(df.head())

ColumnNotFoundError: 5m

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'group_by' <---
 WITH_COLUMNS:
 [col("timestamp").dt.truncate([String(5m)]).alias("time_interval")] 
   WITH_COLUMNS:
   [col("timestamp").dt.truncate([String(5m)]).alias("time_interval")] 
     WITH_COLUMNS:
     [col("latitude0").as_struct([col("longitude0"), col("latitude"), col("longitude")]).map_list().alias("distance")] 
       WITH_COLUMNS:
       [col("latitude").shift([dyn int: 1]).over([col("car_id")]).alias("latitude0"), col("longitude").shift([dyn int: 1]).over([col("car_id")]).alias("longitude0")] 
         WITH_COLUMNS:
         [col("latitude0").as_struct([col("longitude0"), col("latitude"), col("longitude")]).map_list()] 
           WITH_COLUMNS:
           [col("latitude").shift([dyn int: 1]).over([col("car_id")]).alias("latitude0"), col("longitude").shift([dyn int: 1]).over([col("car_id")]).alias("longitude0")] 
             WITH_COLUMNS:
             [col("timestamp").diff().strict_cas